In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.stats import mode,zscore
from matplotlib import pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error,mean_squared_error

data = pd.read_json("../../../../data/data.json")

data= data[data["capture"]== "tw2"]

data.sample(2)

,index,mac,timestamp,rssi,frequency,iq,angle,distance,capture,switch_order,ref,phases,aoa
33886,447,c300000a7a48,2024-11-29 17:41:30.392000+00:00,-50,2402,"[53, -149, 128, 83, -112, 110, -90, -135, 139,...",25,150.0,tw2,2-2-2-0-5-6-14-13-8-10,"[-1.2290484384, 0.5752760180000001, 2.36520325...","[1.8945589509, 0.272570555, -1.3791478301, -2....","[25.5203103799, 24.847555921, 38.2959288061, 5..."
33145,207,c300000a7a48,2024-11-29 17:33:42.147000+00:00,-52,2402,"[-132, -115, 137, -102, 89, 151, -161, 61, -40...",20,150.0,tw2,2-2-2-0-5-6-14-13-8-10,"[-2.4249120327, -0.6399882427, 1.0382044553, 2...","[0.6275389526, -1.5385494444, -3.0242766981, 2...","[13.672241944, 28.6547960058, 36.1709616972, 3..."


# Coluna

Esse modelo usa como entrada cada coluna IQ que sabemos que existe no pacote, logo precisamos gerá-las

In [6]:
#2-2-2-0-5-6-14-13-8-10 = 8*2 elementos em cada coluna, 2*8 de ref
# CTE = 20*8 => 160-4-8= 148 => 148*2 = 74 =>74//8 => 9 com resto 2, logo precisamos ignorar a última amostra 
data["training"] = data["phases"].map(lambda x: [x[i:i+8] for i in range(0,len(x),8)][:-1]) 
len(data["training"].iloc[0])

9

In [7]:
tsamples =[]
tlabels =[]

vsamples=[]
vlabels=[]

# Separamos no dataset para que a gnt seja capaz de depois analizar o conteúdo por pacote
tdataset, vdataset = train_test_split(data,test_size=0.3,random_state=10)

for row in tdataset.iterrows():
    for element in row[1]["training"]:
        tsamples.append(element)
        tlabels.append(row[1]["angle"])

for row in vdataset.iterrows():
    for element in row[1]["training"]:
        vsamples.append(element)
        vlabels.append(row[1]["angle"])

X_train = tsamples
X_val = vsamples


y_train = tlabels
y_val = vlabels 


In [8]:
tv=pd.DataFrame([{"angulo":angle, "treinamento":y_train.count(angle),"validacao" :y_val.count(angle)} for angle in set(y_val)])
tv.set_index("angulo")

,treinamento,validacao
angulo,,
0,9477,4050
1,1809,900
2,1890,819
3,1989,720
4,1980,729
5,1863,846
6,1836,873
7,1827,882
8,1971,738


In [9]:
model = xgb.XGBRegressor(enable_categorical=True)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [10]:
#Making predictions on the test set
predictions = model.predict(X_val)

#Calculating accuracy
mae_model =mean_absolute_error(y_val, predictions)
mse_model =mean_squared_error(y_val, predictions)

print(f"MAE= Modelo: {mae_model}")
print(f"MSE= Modelo: {mse_model}")

MAE= Modelo: 5.382930078949942
MSE= Modelo: 70.42270890393851


In [11]:
aoa_val=[]
results_labels= []
results_values = []
for row in vdataset.iterrows():
    pred=[]
    for element in row[1]["training"]:
        pred.append(model.predict([element])[0])
    results_labels.append((row[1]["angle"]))
    results_values.append(np.mean(pred))
    aoa_val.append(np.mean(row[1]["aoa"]))

In [12]:
#Calculating accuracy

mae_model =mean_absolute_error(results_labels, results_values)
mse_model =mean_squared_error(results_labels, results_values)

mae_aoa =mean_absolute_error(results_labels, aoa_val)
mse_aoa =mean_squared_error(results_labels, aoa_val)

print(f"MAE= Modelo: {mae_model}\t AoA={mae_aoa}")
print(f"MSE= Modelo: {mse_model}\t AoA={mse_aoa}")

MAE= Modelo: 3.6397826482282376	 AoA=17.91555958745627
MSE= Modelo: 33.16256139564386	 AoA=789.3761852881698


In [13]:
model.save_model("./regressor_pd_row.json")